In [74]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import global_add_pool


class DeepMultisets(torch.nn.Module):

    def __init__(self, dim_features, dim_target):
        super(DeepMultisets, self).__init__()

        hidden_units = 32

        self.fc_vertex = Linear(dim_features, hidden_units)
        self.fc_global1 = Linear(hidden_units, hidden_units)
        self.fc_global2 = Linear(hidden_units, dim_target)

    def forward(self, data):
        x, batch = data.x, data.batch

        x = F.relu(self.fc_vertex(x))
        x = global_add_pool(x, batch)  # sums all vertex embeddings belonging to the same graph!
        x = F.relu(self.fc_global1(x))
        x = self.fc_global2(x)
        return x

In [69]:
class DeepMultisets(torch.nn.Module):
    def __init__(self, dim_features, dim_target, batch):
        super(DeepMultisets, self).__init__()

        hidden_units = 32
        
        self.batch = batch
        
        self.fc_vertex = Linear(dim_features, hidden_units)
        self.fc_global1 = Linear(hidden_units, hidden_units)
        self.fc_global2 = Linear(hidden_units, dim_target)

    def forward(self, x):
        x, batch = x, self.batch

        x = F.relu(self.fc_vertex(x))
        x = global_add_pool(x, batch)  # sums all vertex embeddings belonging to the same graph!
        x = F.relu(self.fc_global1(x))
        x = self.fc_global2(x)
        return x

In [70]:
model = DeepMultisets(dim_features=1, dim_target=4, batch=torch.tensor([0,0,0]))

In [71]:
torch.tensor([[1.],[2.],[3.]]).float().shape

torch.Size([3, 1])

In [72]:
model(torch.tensor([[1.],[2.],[3.]]).float())

tensor([[-0.1550, -0.9311,  0.1201, -0.2438]], grad_fn=<AddmmBackward>)

In [73]:
model(torch.tensor([[2.],[1.],[3.]]).float())

tensor([[-0.1550, -0.9311,  0.1201, -0.2438]], grad_fn=<AddmmBackward>)

In [30]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data, DataLoader

from sklearn.model_selection import KFold
from torch.utils.data.dataset import Subset
from torch_geometric.utils import to_networkx

import networkx as nx


for name in ["COLLAB"]:
    print(name)
    dataset = TUDataset(root='data/TUDataset', name=name)

COLLAB


In [17]:
fold = KFold(
    n_splits=10, shuffle=True, random_state=0
)

In [18]:
for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(dataset, dataset.data.y)):
        train_loader = DataLoader(
            dataset[[i for i in train_idx]],
            shuffle=True,
            batch_size=32,
        )
        valid_loader = DataLoader(
            dataset[[i for i in valid_idx]],
            shuffle=False,
            batch_size=32,
        )
        break

In [31]:
def make_x(data):
    for num in range(len(data.y)):
        G = to_networkx(data[num])
        new_x = torch.tensor(
                [[i] for i in list(dict(nx.degree(G)).values())],
                dtype=torch.float,
            )
        if num == 0:
            x = new_x
        else:
            x = torch.cat([x, new_x])
    return x

In [32]:
for data in train_loader:
    break

In [36]:
data.x = make_x(data)

In [65]:
data.batch

tensor([ 0,  0,  0,  ..., 31, 31, 31])

In [41]:
model(data).shape

torch.Size([32, 4])